In [1]:
from pymongo import MongoClient
from bson import ObjectId
from collections import defaultdict
import pandas as pd
import scipy.sparse
import numpy as np
import time

Mclient  = MongoClient('URL')
moodsDb = Mclient['moods_dev']

In [49]:
##################################### create a matrix of articles vs article's categories #####################

# articleName = moodsDb.user_articles.find({}).limit(100000)
# articleID_Categories = defaultdict(set)
# for row in articleName:
#     i = 10
#     articleID = str(row.get('_id'))
#     for cate in row["categories"]:
#         articleCategories = str(cate)
#         articleID_Categories[articleID].add(articleCategories)

##################################### create a matrix of UserID vs article's categories of articles read by user #####################
#MOSHE - 59c260e93f75762a16a851bf
# userID_Categories = defaultdict(list)
# for usr in userID:
#     articlesRead = moodsDb.users.find({'_id':ObjectId(usr)})
#     for user in articlesRead:
#         articleIDS = user['articlesread']
#         if len(articleIDS) == 0:
#             break
#         else:
#             for artRead in articleIDS:
#                 artID = str(artRead)
#                 if articleID_Categories[artID]:
#                     for x in list(articleID_Categories[artID]):
#                         if x not in userID_Categories[usr]:
#                             userID_Categories[usr].append(x)
#                         else:
#                             continue
#                 else:
#                     continue
##################################### create a matrix of UserID vs articles read by user #####################
# userID_Articles = defaultdict(list)
# for usr in userID:
#     articlesRead = moodsDb.users.find({'_id':ObjectId(usr)})
#     for user in articlesRead:
#         articleIDS = user['articlesread']
#         if len(articleIDS) == 0:
#             break
#         else:
#             for artRead in articleIDS:
#                 artID = str(artRead)
#                 if artID not in userID_Articles[usr]:
#                     userID_Articles[usr].append(artID)
#                 else:
#                     continue

In [50]:
# df2 = pd.DataFrame.from_dict(articleID_Categories, orient='index')
# df2 = df2.stack().reset_index()
# df2.level_1=1
# df2.pivot(index='level_0',columns=0,values='level_1').fillna(0)
# df2.to_csv('articleID_Category.csv', sep=',', encoding='utf-8')

# df3 = pd.DataFrame.from_dict(userID_Categories, orient='index')
# df3 = df3.stack().reset_index()
# df3.level_1=1
# df3.pivot(index='level_0',columns=0,values='level_1').fillna(0)
# df3.to_csv('UserID_Category.csv', sep=',', encoding='utf-8')

# df4 = pd.DataFrame.from_dict(userID_Articles, orient='index')
# df4 = df4.stack().reset_index()
# df4.level_1=1
# #df4.pivot(index='level_0',columns=0,values='level_1').fillna(0)
# art_user = pd.DataFrame(df4.pivot(index='level_0',columns=0,values='level_1').fillna(0).to_records())
#art_user.to_csv('UserID_ArticlesRead.csv', sep=',', encoding='utf-8',index=False)

In [2]:
### Get the Number of Users IDs who has read an articles
userArticleCategories = moodsDb.users.find({'articlesread':{'$exists':True}})
userID = []
for a in userArticleCategories:
    userID.append(str(a.get('_id')))

In [3]:
######## Create a list of total articles id #################
totalArticles = []
articleName = moodsDb.user_articles.find({})
for art in articleName:
    totalArticles.append(str(art['_id']))

In [4]:
######### Create a dictionary for keys as userID and values as article read by the userID
userID_Articles = defaultdict(list)
for usr in userID:
    articlesRead = moodsDb.users.find({'_id':ObjectId(usr)})
    for user in articlesRead:
        articleIDS = user['articlesread']
        if len(articleIDS) == 0:
            break
        else:
            for artRead in articleIDS:
                artID = str(artRead)
                if artID not in userID_Articles[usr]:
                    userID_Articles[usr].append(artID)
                else:
                    continue

In [7]:
userID_art_ratings = {}
optimumScore = int()
articlesRead = []
readArticle = 0
UnreadArticle = 0
start = time.clock()
for article in totalArticles:
    for usrKey,artVal in userID_Articles.items():
        if article in artVal:
            articleEvaluation = moodsDb.user_ratings.find({'$and':[{'user_id':ObjectId(usrKey)},{'article_id':ObjectId(article)}]})
            for row in articleEvaluation:
                OpSum = sum(row['opinion'][op] for op in row['opinion'])
                ExpSum = sum(row['experience'][exp] for exp in row['experience'])
                optimumScore = round((OpSum+ExpSum)/70,2)
            if optimumScore < 0.35:
                optimumScore = -1
                user_id,article_id,article_score = (usrKey,article,optimumScore)
                if user_id in userID_art_ratings:
                    userID_art_ratings[user_id][article_id] = article_score
                else:
                    userID_art_ratings[user_id] = {article_id: article_score}
            elif optimumScore >= 0.35:
                optimumScore = 1
                user_id,article_id,article_score = (usrKey,article,optimumScore)
                if user_id in userID_art_ratings:
                    userID_art_ratings[user_id][article_id] = article_score
                else:
                    userID_art_ratings[user_id] = {article_id: article_score}
            else:
                optimumScore = 0.0
                user_id,article_id,article_score = (usrKey,article,optimumScore)
                if user_id in userID_art_ratings:
                    userID_art_ratings[user_id][article_id] = article_score
                else:
                    userID_art_ratings[user_id] = {article_id: article_score}
                break
        else:
            optimumScore = 0.0
            user_id,article_id,article_score = (usrKey,article,optimumScore)
            if user_id in userID_art_ratings:
                userID_art_ratings[user_id][article_id] = article_score
            else:
                userID_art_ratings[user_id] = {article_id: article_score}
                
print(time.clock() - start)

10.438759000000005


In [113]:
UserBasedarticleRating = pd.DataFrame(userID_art_ratings).transpose()

In [114]:
UserBasedarticleRating.fillna(0,inplace=True)
user_id = pd.Series(UserBasedarticleRating.index.values)
UserBasedarticleRating = UserBasedarticleRating.values

In [10]:
user_id = pd.Series(UserBasedarticleRating.index.values)
articleID = pd.Series(UserBasedarticleRating.dtypes.index.values)
articles_ID = pd.DataFrame(articleID,columns=['article_ID'])

In [11]:
UserRatedArticle = scipy.sparse.csr_matrix(UserBasedarticleRating)

In [12]:
############################ Why Model is not compatible for Collabrative Filtering #########################
# matrix_size = UserRatedArticle.shape[0]*UserRatedArticle.shape[1] # Number of possible interactions in the matrix
# num_purchases = len(UserRatedArticle.nonzero()[0]) # Number of items interacted with
# sparsity = 100*(1 - (num_purchases/matrix_size))
# sparsity

In [27]:
train = UserRatedArticle[:11]
test = UserRatedArticle[10:21]

In [62]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
#for x in [0.1,0.2,0.02,0.03,0.04,0.5,1]:
model = LightFM(no_components=20,learning_rate=0.01, loss='warp')

model.fit_partial(UserRatedArticle, epochs=10)
# train_precision = precision_at_k(model, train,k=5).mean()
# test_precision = precision_at_k(model, test,k=5).mean()

# train_auc = auc_score(model, train).mean()
# test_auc = auc_score(model, test,train_interactions=train).mean()

# print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
# print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

In [63]:
n_users, n_articles = UserRatedArticle.shape

In [44]:
user_id[user_id == '58ca9a4d39b93c16a0512b16'].index

Int64Index([0], dtype='int64')

In [65]:
scores = model.predict(user_id[user_id == '591ae23171273548d31939a0'].index, np.arange(n_articles))
article_score = reversed(sorted(scores))
top_items = articles_ID['article_ID'][np.argsort(-scores)]
for x in top_items[:5]:
    print("article ID %s" % x)
for x in list(article_score)[:5]:
    print("article Score",x)

AssertionError: 

In [13]:
lambda_ = 0.1
n_factors = 100
m, n = UserBasedarticleRating.shape
n_iterations = 5

X = 5 * np.random.rand(m, n_factors)
Y = 5 * np.random.rand(n_factors, n)

In [111]:
print(UserBasedarticleRating)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [34]:
W = UserBasedarticleRating != 0.0
W[W == True] = 0
W[W == False] = 1
# To be consistent with our Q matrix
W = W.astype(np.float64, copy=False)

/home/doodlews-98/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:214: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  ", try using != instead.", SparseEfficiencyWarning)
/home/doodlews-98/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [35]:
W_array = W.toarray()

In [28]:
def get_error(UserBasedarticleRating, X, Y, W):
    return np.sum((W * (UserBasedarticleRating - np.dot(X, Y)))**2)

In [37]:
weighted_errors = []
for ii in range(n_iterations):
    for u, Wu in enumerate(W_array):
        X[u] = np.linalg.solve(np.dot(Y, np.dot(np.diag(Wu), Y.T)) + lambda_ * np.eye(n_factors),
                               np.dot(Y, np.dot(np.diag(Wu), UserBasedarticleRating[u].T))).T
    for i, Wi in enumerate(W_array.T):
        Y[:,i] = np.linalg.solve(np.dot(X.T, np.dot(np.diag(Wi, X)) + lambda_ * np.eye(n_factors),
                                 np.dot(X.T, np.dot(np.diag(Wi, UserBasedarticleRating[:, i])))
    weighted_errors.append(get_error(UserBasedarticleRating, X, Y, W_array))
    print('{}th iteration is completed'.format(ii))

SyntaxError: invalid syntax (<ipython-input-37-28b6ee969d69>, line 9)

In [59]:
weighted_Q_hat = np.dot(X,Y)

In [116]:
def print_recommendations(W=W, Q=UserBasedarticleRating, Q_hat=None, article_titles=totalArticles):
    Q_hat -= np.min(Q_hat)
    Q_hat *= float(1) / np.max(Q_hat)
    article_ids = np.argmax(Q_hat - 1 * W, axis=1)
    for jj, article_id in zip(range(m), article_ids):
        article_id = list(np.array(article_id).reshape(-1,))
        print('User {} liked {}\n'.format(jj + 1, ', '.join([article_titles[ii] for ii, qq in enumerate(Q[jj]) if qq > 0])))
        print('User {} did not like {}\n'.format(jj + 1, ', '.join([article_titles[ii] for ii, qq in enumerate(Q[jj]) if qq < 0])))
        print('\n User {} recommended movie is {} - with predicted rating: {}'.format(jj + 1, article_titles[article_id[0]], Q_hat[jj, article_id]))
        print('\n' + 100 *  '-' + '\n')

In [117]:
print_recommendations(Q_hat=weighted_Q_hat)

User 1 liked 5a0c1c80e94bff22c605c6bc

User 1 did not like 59c38e3adb95281cb8534393, 59c38eb6db95281cb853446c, 59c38ec0db95281cb853447e, 59d6216ddb95281f20f14060, 59d6216ddb95281f20f14061, 59d62187db95281f20f1408d, 59d621bfdb95281f20f140e4, 59d621c0db95281f20f140e8, 59d621c6db95281f20f140f4, 59d76e92db952815549f75b1, 59d76e92db952815549f760d, 59d76e92db952815549f7610, 59d76e92db952815549f7924, 59f473f2db95281a0c844756, 59f473f2db95281a0c8447d3, 59f473f2db95281a0c844835, 59f473f2db95281a0c844846, 59f473f2db95281a0c844863, 59f473f2db95281a0c844936, 59f473f2db95281a0c84493c, 5a02e724e94bff388ad7f61c, 5a099481e94bff17160fac98, 5a099481e94bff17160fac99, 5a099481e94bff17160fac9a, 5a099481e94bff17160fac9b, 5a0bf82edb95280eb4bd0b67, 5a0c12dfe94bff22c6056661, 5a0c12f0e94bff22c605671d, 5a0c1310e94bff22c6056859, 5a0c1a3ce94bff22c605b065, 5a537040e94bff769b142028, 5a537040e94bff769b14202a, 5a537040e94bff769b14202b, 5a537040e94bff769b14202c, 5a537040e94bff769b14202d, 5a537040e94bff769b14202e, 5a537

User 11 liked 59d6216edb95281f20f14063, 5a537040e94bff769b142090, 5a6724953bd326167db5a482

User 11 did not like 59c38e3edb95281cb8534399, 59c38e3edb95281cb853439a, 59c38e4adb95281cb85343af, 59c38e54db95281cb85343c1, 59c38ea1db95281cb8534447, 59d62155db95281f20f14038, 59d62156db95281f20f1403a, 59d6215ddb95281f20f14043, 59d6216ddb95281f20f14060, 59d6216ddb95281f20f14061, 59d6216edb95281f20f14062, 59d6216fdb95281f20f14064, 59d6216fdb95281f20f14065, 59d62170db95281f20f14066, 59d62170db95281f20f14067, 59d62171db95281f20f14068, 59d62171db95281f20f14069, 59d62172db95281f20f1406a, 59d62172db95281f20f1406b, 59d62173db95281f20f1406c, 59d62173db95281f20f1406d, 59d621c6db95281f20f140f4, 59d623aedb95281f20f14406, 59d76e92db952815549f74b9, 59d76e92db952815549f75be, 59d76e92db952815549f77ab, 59edb7a2892faf63ec207a5f, 59f1ce02db952821308ba85f, 59f473f2db95281a0c84476c, 59f473f2db95281a0c84478f, 59f473f2db95281a0c8447a6, 59f473f2db95281a0c8447aa, 59f473f2db95281a0c8447c7, 59f473f2db95281a0c8447d3, 59f

User 21 did not like 5aa910a5b93ba8616645bfae, 5aaa126253b5a02ed4709326


 User 21 recommended movie is 5a2645d8e94bff681e98dd17 - with predicted rating: [1.]

----------------------------------------------------------------------------------------------------

User 22 liked 

User 22 did not like 5aacbe70e94bff5994fb0699, 5aacc5aa514e384f03bcbc69


 User 22 recommended movie is 5a9d10e1e94bff1ea2dd56c9 - with predicted rating: [0.83619394]

----------------------------------------------------------------------------------------------------



In [115]:
user_id

0     58ca9a4d39b93c16a0512b16
1     5922bb5eca31e823d89917bf
2     592d2735ca31e823d89917e1
3     598f02a4295deb5eef22fd47
4     599d818fcb5b0e5a2d19ce47
5     59aeb9103eff8004a2f73987
6     59c2598681d14d26cb0e920b
7     59c260e93f75762a16a851bf
8     59c35b2ccf2edc3b8e5f1c2d
9     59ddb825b0a06048405c0d8d
10    59f02efcc32ec00ec67662ec
11    59fc42d0e33fa16647db78f5
12    5a00583223d9af1b04a736b7
13    5a053273cc2d0b07b601120e
14    5a400b721831cb024a8c5e24
15    5a559e3c139d1a6bcad3a860
16    5a8526643879b4731f4bc1e6
17    5a8fb91139b45e1e1c8f535b
18    5aa23274a9541c7f598c5158
19    5aacdf778c36236660e00acd
20    5ab1e1663e32197a9c8085dc
21    5ab220046afc212f6fb74eed
dtype: object